In [20]:
import numpy as np                            # np making numpy arrey
import pandas as pd                           #storing data in a DataFrame
import re                                     #  Re : regular expresion to search words in a  document
from nltk.corpus import stopwords             # stopwords  stop the words which does not add much value to te document(a, an ,and ,..,)
from nltk.stem.porter import PorterStemmer   ## stem returns the root of the word
from sklearn.feature_extraction.text import TfidfVectorizer   #TfidfVectorizer converts text to features vectors which are numbers
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
import nltk
nltk.download('stopwords') 

[nltk_data] Downloading package stopwords to /home/scg/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [62]:
print (stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

## Data Preprocessing 

In [63]:
news_dataset= pd.read_csv('train.csv')

In [64]:
news_dataset.shape

(20800, 5)

In [65]:
news_dataset.head()  # labels are :  1 or 0

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [66]:
# check if some values are missed?   =======>  .isnull().sum()
news_dataset.isnull()
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [67]:
#replacing the null values with empty string ======> .fillna('')
news_dataset = news_dataset.fillna('')

In [68]:
news_dataset.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [69]:
# creat a new coulm named contanet and merging the atuer name and news title 
news_dataset['content']= news_dataset['author']=news_dataset['title']
                                                            

In [70]:
print(news_dataset['content'])

0        House Dem Aide: We Didn’t Even See Comey’s Let...
1        FLYNN: Hillary Clinton, Big Woman on Campus - ...
2                        Why the Truth Might Get You Fired
3        15 Civilians Killed In Single US Airstrike Hav...
4        Iranian woman jailed for fictional unpublished...
                               ...                        
20795    Rapper T.I.: Trump a ’Poster Child For White S...
20796    N.F.L. Playoffs: Schedule, Matchups and Odds -...
20797    Macy’s Is Said to Receive Takeover Approach by...
20798    NATO, Russia To Hold Parallel Exercises In Bal...
20799                            What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [71]:
# seperating the data and labels  =====> .drop('labels',axis=1)
x= news_dataset .drop('label',axis=1)
y= news_dataset['label']

In [72]:
#print(x)
#print(y)

In [73]:
# stemin: steming is the process to reduce a word to its root word

In [74]:
port_stem= PorterStemmer()

In [75]:
#('[^a-zA-Z]',' ',content)
#sub : call library to search , 
# '[^a-zA-Z] we just need alphabets
# stemmed_content.lower()  convert all leters to lower case
# port_stem.stem stem all the word not in the stopwords

def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content              

In [76]:
# apply def on dataset =====> .apply(stemming)
news_dataset['content']= news_dataset['content'].apply(stemming)

In [77]:
print(news_dataset['content'])

0        hous dem aid even see comey letter jason chaff...
1          flynn hillari clinton big woman campu breitbart
2                                     truth might get fire
3                 civilian kill singl us airstrik identifi
4        iranian woman jail fiction unpublish stori wom...
                               ...                        
20795            rapper trump poster child white supremaci
20796      n f l playoff schedul matchup odd new york time
20797    maci said receiv takeov approach hudson bay ne...
20798             nato russia hold parallel exercis balkan
20799                                          keep f aliv
Name: content, Length: 20800, dtype: object


In [78]:
#seperating data and label
x= news_dataset['content'].values
y= news_dataset['label'].values

In [79]:
print(x)
print(y)

['hous dem aid even see comey letter jason chaffetz tweet'
 'flynn hillari clinton big woman campu breitbart' 'truth might get fire'
 ... 'maci said receiv takeov approach hudson bay new york time'
 'nato russia hold parallel exercis balkan' 'keep f aliv']
[1 0 1 ... 0 1 1]


In [80]:
y.shape

(20800,)

In [81]:
# convert the words to numerical data
# TfidfVectorizer() counts the nemuber of times words in the text
vectorizer= TfidfVectorizer()
vectorizer.fit(x)
x=vectorizer.transform(x)

In [82]:
print(x)

  (0, 13610)	0.3204429041871619
  (0, 11663)	0.28865071055944214
  (0, 7472)	0.3286263634991149
  (0, 6751)	0.37807503658407626
  (0, 6143)	0.24709678286281997
  (0, 4394)	0.2624700707018907
  (0, 3340)	0.3043365499413147
  (0, 2578)	0.2776884546129286
  (0, 2165)	0.4135903166037093
  (0, 243)	0.3038505731698672
  (1, 14597)	0.4029069338022535
  (1, 5974)	0.255243301322185
  (1, 4874)	0.49350006801559954
  (1, 2444)	0.25583271340004526
  (1, 1946)	0.5127916215485694
  (1, 1650)	0.21311459271319796
  (1, 1313)	0.3938922297906677
  (2, 13543)	0.520272613528009
  (2, 8285)	0.5879029381082882
  (2, 5278)	0.41416349493213955
  (2, 4770)	0.460603020443823
  (3, 13947)	0.2910988442735044
  (3, 12016)	0.4595551330231538
  (3, 7080)	0.3129596268768328
  (3, 6271)	0.48520717569885846
  :	:
  (20796, 14760)	0.12072790494289075
  (20796, 13246)	0.11914995894745603
  (20796, 11529)	0.4694159922014375
  (20796, 9959)	0.4804399319541416
  (20796, 9158)	0.4804399319541416
  (20796, 8881)	0.11669800295

In [92]:
#split data to train and test data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, stratify=y, random_state=2)

In [93]:
# train the model
model= LogisticRegression()


In [101]:
model.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [107]:
#accuracy  score on the training data
x_train_prediction=model.predict(x_train)
training_accuracy= accuracy_score(x_train_prediction, y_train )

In [104]:
print(training_accuracy)

0.9399038461538461


In [112]:
x_test_prediction=model.predict(x_test)

In [113]:
test_accuracy= accuracy_score(x_test_prediction, y_test )

In [114]:
print(test_accuracy)

0.9298076923076923


In [121]:
# make a pridiction
x_newdata= x_test[1]
prediction= model.predict(x_newdata)
print(prediction)


[0]


In [122]:
# check the accuracy
print(y_test[1])

0
